In [ ]:
# A SYNCHRONOUS implementation of how a FSM works to generate
# suggestions for behavior.
def syncronous_behavior(car, loc, M, predictor, fsm):
    if not fsm.initialized:
        # set finite state machine to initial state
        fsm.set_initial_state()
    
    # loop until driving is done
    while not car.has_reached_goal():
        car.state = loc.get_localization() # car's state is (lane, s, d, yaw, s_dot, d_dot, sigma_s, ...)
        
        # fsm state is one of small number of options
        fsm.state = fsm.get_state(car.state) 
        
        # get predictions for what nearby vehicles will do
        next_predictions = predictor.get_predictions() 
        
        # pare down the possible next states.
        possible_next_states = fsm.get_possible_states_after(car.state)  
        
        # calculate the cost associated w/ each state
        state_costs = []
        for state in possible_next_states:
            cost = fsm.get_cost(state, car, M, next_predictions)
            state_costs.append((cost, state))
        
        # take the state with the lowest cost
        best_cost, best_state = min(state_costs, key=lambda x: x.0)
        
        # generate a suggestion for the best state
        suggestion = fsm.generate_best_behavior(best_state, car, M, next_predictions)
        fsm.suggest_next_state(best_state)

def get_cost(state, car, M, next_predictions):
    cost = 0
    for cf in [lane_change_cost, velocity_cost, gap_size_cost]:
        cost += cf(state, car, M, next_predictions)
    return cost
    
    

In [ ]:
class AsyncModule(object):
    
    

In [1]:
class PubSubBehaviorPlanner(object):
    
    _initial_state = "ready"
    
    _states = []
    
    _allowed_state_transitions = [
        [],
    ]
    
    def __init__(self, _map):
        self._map = _map
        self._current_state = self._initial_state
        self._location = None
        self._predictions = None
        self._suggested_next_state = None

    def _onLocalizationUpdate(self, data):
        """
        Called when localizationUpdate event is emitted
        """
        pass
    
    def _onPredictionUpdate(self, data):
        """
        Called when predictionUpdate event is emitted
        """
        pass
    
    def _onGoalChange(self, data):
        """
        Called when predictionUpdate event is emitted
        """
        pass
    
    def _emit(self, event, data):
        """
        Broadcast point for all events
        
        """
    
    def handle(self, event, data):
        """
        Entry point for all events which calls the appropriate handler.
        """
        handlers = {
            "localizationUpdate" : self._onLocalizationUpdate,
            "predictionUpdate"   : self._onPredictionUpdate,
            "goalChange"         : self._onGoalChange,
        }
        handler = handlers.get(event)
        if not handler: 
            print "event {} not recognized".format(event)
            return
        handler(data)
    
    def request_behavior(self, time_horizon=3.0, within_time=0.4):
        """
        Called by planning module to indicate preferences 
        for next behavior update.
        """
    